In [10]:
import os 
from pathlib import Path
os.chdir(r"d:\networkbot")


In [11]:
%pwd

'd:\\networkbot'

In [12]:
#split data into chunks 
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 


In [13]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents



In [14]:
extracted_data=load_pdf_file(data='Data/')

In [15]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [16]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 6141


In [17]:
from langchain.embeddings import HuggingFaceEmbeddings

In [18]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [19]:
embeddings = download_hugging_face_embeddings()

C:\Users\win 10\AppData\Local\Temp\ipykernel_11776\1196424635.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\win 10\miniconda3\envs\chatbot1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))
#query_result

Length 384


In [21]:
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')
api_base=os.environ.get('openai.api_base')

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "bot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [24]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [25]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

KeyboardInterrupt: 

In [26]:
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [27]:
docsearch

In [28]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [29]:
retrieved_docs = retriever.invoke("What is switch?")

In [30]:
retrieved_docs

[Document(id='9b4a8d5f-79e7-414f-99c0-4dad24984c4d', metadata={'author': 'Peter L Dordal', 'creationdate': '2023-07-20T19:58:26-05:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2023-07-20T19:58:26-05:00', 'page': 60.0, 'page_label': '49', 'producer': 'pdfTeX-1.40.20', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.20 (TeX Live 2019/Debian) kpathsea version 6.3.1', 'source': 'Data\\ComputerNetworks1.pdf', 'subject': '', 'title': 'An Introduction to Computer Networks', 'total_pages': 967.0, 'trapped': '/False'}, page_content='other ports.\nOriginally, switches were seen as providing interconnection (“bridging”) between separate physical Ether-\nnets; a switch for such a purpose needed just two ports. Later, a switched Ethernet was seen as one large\n“virtual” Ethernet, composed of smaller collision domains. Although the term “switch” is now much more\ncommon than “bridge”, the latter is still in use, particularly by the IEEE. For some, a switch is a 

In [31]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
from groq import Groq
client = Groq(api_key="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")  

response = client.chat.completions.create(
    messages=[{"role": "user", "content": "What is a computer tcp /ip ?"}],
    model="llama3-70b-8192",  
    temperature=0.7, 
)

print(response.choices[0].message.content)


TCP/IP (Transmission Control Protocol/Internet Protocol) is a fundamental communication protocol suite used to interconnect devices on the internet and facilitate communication between them. It's the backbone of the internet, allowing devices to communicate with each other and exchange data.

Here's a breakdown of the two main components:

**TCP (Transmission Control Protocol)**:

TCP is a transport-layer protocol that ensures reliable, error-checked, and sequential delivery of data between devices. It's responsible for:

1. **Connection establishment**: Setting up a connection between the sender and receiver before data transfer begins.
2. **Segmentation and reassembly**: Breaking down data into smaller packets (segments) and reassembling them at the receiving end.
3. **Error detection and correction**: Checking for errors during transmission and retransmitting corrupted packets.
4. **Flow control**: Regulating the amount of data sent at one time to prevent network congestion.

**IP (

In [33]:
from langchain_community.llms import Groq

llm = Groq(api_key="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", model_name="llama3-70b-8192")

response = llm.invoke("Tell me a joke.")
print(response)

ImportError: cannot import name 'Groq' from 'langchain_community.llms' (c:\Users\win 10\miniconda3\envs\chatbot1\lib\site-packages\langchain_community\llms\__init__.py)

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(api_key="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", model="llama3-70b-8192")


Here's one:

Why couldn't the bicycle stand up by itself?

(Wait for it...)

Because it was two-tired!

Hope that made you laugh!
